# Main imports and code - Preparation

In [4]:
# check which gpu we're using
!nvidia-smi

Mon Mar  4 21:50:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
!pip install simpletransformers
!pip install tensorboardx
!pip install googletrans
!pip install nltk
!pip install accelerate -U
!pip install transformers[torch]

  Using cached simpletransformers-0.70.0-py3-none-any.whl (315 kB)
  Using cached datasets-2.18.0-py3-none-any.whl (510 kB)
  Using cached seqeval-1.2.2-py3-none-any.whl
  Using cached wandb-0.16.3-py3-none-any.whl (2.2 MB)
  Using cached streamlit-1.31.1-py2.py3-none-any.whl (8.4 MB)
  Using cached GitPython-3.1.42-py3-none-any.whl (195 kB)
  Using cached sentry_sdk-1.40.6-py2.py3-none-any.whl (258 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached setproctitle-1.3.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30 kB)
  Using cached dill-0.3.8-py3-none-any.whl (116 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
  Using cached validators-0.22.0-py3-none-any.whl (26 kB)
  Using cached pydeck-0.8.1b0-py2.py3-none-any.whl (4.8 MB)
  Using cached watchdog-4.0.0-py3-none-manylinux2014_x86_64.whl (82 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl (62 kB)
  Using cached smmap-5.0.1-p

In [6]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
from urllib import request
import pandas as pd
import logging
import torch
from collections import Counter
from ast import literal_eval

In [7]:
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# check gpu
cuda_available = torch.cuda.is_available()

print('Cuda available? ',cuda_available)

Cuda available?  True


In [8]:
if cuda_available:
  import tensorflow as tf
  # Get the GPU device name.
  device_name = tf.test.gpu_device_name()
  # The device name should look like the following:
  if device_name == '/device:GPU:0':
      print('Found GPU at: {}'.format(device_name))
  else:
      raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


## Fetch Don't Patronize Me! data manager module

In [9]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [10]:
#the raw dataset
!wget -O dontpatronizeme_pcl.tsv https://raw.githubusercontent.com/CRLala/NLPLabs-2024/main/Dont_Patronize_Me_Trainingset/dontpatronizeme_pcl.tsv

--2024-03-04 21:51:02--  https://raw.githubusercontent.com/CRLala/NLPLabs-2024/main/Dont_Patronize_Me_Trainingset/dontpatronizeme_pcl.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3122842 (3.0M) [text/plain]
Saving to: ‘dontpatronizeme_pcl.tsv’

dontpatronizeme_pcl 100%[===================>]   2.98M  --.-KB/s    in 0.04s   

2024-03-04 21:51:02 (74.1 MB/s) - ‘dontpatronizeme_pcl.tsv’ saved [3122842/3122842]



In [11]:
# their training selection set - i assume it's for picking up corresponding id from the raw dataset
!wget -O train_semeval_parids-labels.csv https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/practice%20splits/train_semeval_parids-labels.csv

--2024-03-04 21:51:02--  https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/practice%20splits/train_semeval_parids-labels.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 241839 (236K) [text/plain]
Saving to: ‘train_semeval_parids-labels.csv’

train_semeval_parid 100%[===================>] 236.17K  --.-KB/s    in 0.02s   

2024-03-04 21:51:02 (12.0 MB/s) - ‘train_semeval_parids-labels.csv’ saved [241839/241839]



In [12]:
#their testing selection set
!wget -O dev_semeval_parids-labels.csv https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/practice%20splits/dev_semeval_parids-labels.csv

--2024-03-04 21:51:03--  https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/practice%20splits/dev_semeval_parids-labels.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61151 (60K) [text/plain]
Saving to: ‘dev_semeval_parids-labels.csv’

dev_semeval_parids- 100%[===================>]  59.72K  --.-KB/s    in 0.007s  

2024-03-04 21:51:03 (8.36 MB/s) - ‘dev_semeval_parids-labels.csv’ saved [61151/61151]



In [13]:
#testing set
!wget -O task4_test.tsv https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/TEST/task4_test.tsv

--2024-03-04 21:51:03--  https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/TEST/task4_test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1145277 (1.1M) [text/plain]
Saving to: ‘task4_test.tsv’

task4_test.tsv      100%[===================>]   1.09M  --.-KB/s    in 0.03s   

2024-03-04 21:51:03 (33.0 MB/s) - ‘task4_test.tsv’ saved [1145277/1145277]



In [14]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

In [15]:
from dont_patronize_me import DontPatronizeMe

In [16]:
dpm = DontPatronizeMe('.', 'task4_test.tsv')

In [17]:
dpm.load_task1()
#Paragraphs with original labels of 0 or 1 are considered to be negative examples of PCL and will have the label 0 = negative.
#Paragraphs with original labels of 2, 3 or 4 are considered to be positive examples of PCL and will have the label 1 = positive.

## Load paragraph IDs

In [18]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')

In [19]:
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

In [20]:
data=dpm.train_task1_df

In [21]:
data


,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0
...,...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0
10466,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3
10467,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4


In [22]:
data = data.dropna()

# RAW Dataset - Q1.a & b Data Analysis Code - SAVE FOR MICHAEL

In [23]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# label_distribution = data['label'].value_counts()
# label_distribution.index = label_distribution.index.astype(str)
# orig_label_distribution = data['orig_label'].value_counts()
# total = label_distribution.sum()

# # distribution of 'label'
# plt.figure(figsize=(3.5, 0.75))
# barplot = sns.barplot(x=label_distribution.values, y=label_distribution.index, orient='h', color='lightblue')
# plt.title('')
# plt.xlabel('')
# plt.ylabel('')
# plt.xticks([])
# for p in barplot.patches:
#     width = p.get_width()
#     count = int(width)
#     percentage = f"({count / total:.1%})"
#     plt.text(width + 1, p.get_y() + p.get_height() / 2, f"{count} {percentage}", ha='left', va='center', color='black')
# for spine in plt.gca().spines.values():
#     spine.set_visible(False)
# plt.gca().tick_params(left=False)  # Remove y-axis tick marks
# plt.tight_layout()  # Adjust the layout to make sure everything fits without overlapping
# plt.savefig('distribution_of_pcl.pdf', format='pdf', bbox_inches='tight')  # Save the figure as a PDF
# plt.show()

# # distribution of 'orig_label'
# agg_data = data.groupby('orig_label').agg(
#     count=('orig_label', 'size'),
#     most_common_label=('label', lambda x: x.mode()[0])
# ).reset_index()
# agg_data['label_group'] = agg_data['most_common_label'].map({0: 'No PCL', 1: 'PCL'})
# color_map = {'No PCL': 'lightblue', 'PCL': 'salmon'}
# plt.figure(figsize=(3.5, 3.5))
# sns.barplot(x='orig_label', y='count', hue='label_group', dodge=False,
#             palette=color_map, data=agg_data)
# plt.title('Distribution of Original Labels')
# plt.xlabel('Original Label')
# plt.ylabel('')
# legend = plt.legend(title='')
# for spine in plt.gca().spines.values():
#     spine.set_visible(False)
# plt.tick_params(axis='x', which='both', bottom=False, top=False)
# legend.get_frame().set_edgecolor('none')  # This removes the border
# plt.tight_layout()
# plt.savefig('distribution_of_orig_labels_by_group.pdf', format='pdf', bbox_inches='tight')
# plt.show()

# # PCL by vulnerable group
# keyword_label_counts = data.groupby(['keyword', 'label']).size().unstack(fill_value=0)
# keyword_label_proportions = keyword_label_counts.div(keyword_label_counts.sum(axis=1), axis=0)
# keyword_label_proportions.reset_index(inplace=True)
# fig, ax = plt.subplots(figsize=(3.5, 2.75))
# ax.barh(keyword_label_proportions['keyword'], keyword_label_proportions[0], label='No PCL', color='lightblue')
# ax.barh(keyword_label_proportions['keyword'], keyword_label_proportions[1], left=keyword_label_proportions[0], label='PCL', color='salmon')
# ax.set_ylabel('')
# ax.set_xlabel('Proportion')
# ax.set_title('')
# ax.tick_params(axis='y', which='both', left=False)  # Remove y-axis tick marks
# legend = ax.legend(title='', loc='lower center', bbox_to_anchor=(0.5, 1), ncol=2)
# legend.get_frame().set_linewidth(0.0)  # Remove the legend border
# for spine in ax.spines.values():
#     spine.set_visible(False)
# plt.tight_layout()
# plt.savefig('pcl_by_keyword.pdf', format='pdf', bbox_inches='tight')
# plt.show()

# # Box plot of paragraph length by PCL label
# data['text_length'] = data['text'].apply(len)
# data['label_str'] = data['label'].map({0: 'No PCL', 1: 'PCL'})
# palette_colors = {'No PCL': 'lightblue', 'PCL': 'salmon'}
# plt.figure(figsize=(3.5, 2.75))  # Adjust the figure size as needed
# ax = sns.boxplot(x='text_length', y='label_str', hue='label_str', data=data, orient='h',
#                  palette={'No PCL': 'lightblue', 'PCL': 'salmon'}, dodge=False, legend=False)
# plt.title('')
# plt.xlabel('Text length')
# plt.ylabel('')
# for spine in ax.spines.values():
#     spine.set_visible(False)
# ax.set_xlim(0, 2000)
# ax.tick_params(axis='y', which='both', left=False)  # Remove y-axis tick marks
# plt.tight_layout()
# plt.savefig('pcl_by_length.pdf', format='pdf', bbox_inches='tight')
# plt.show()



## Get the raw training set (Task 1) - without split

In [24]:
rows = [] # will contain par_id, label and text
for idx in range(len(trids)):
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })


In [25]:
import random

In [26]:
trdf1 = pd.DataFrame(rows)

In [27]:
trdf1

,par_id,community,text,label
0,4341,poor-families,"The scheme saw an estimated 150,000 children f...",1
1,4136,homeless,Durban 's homeless communities reconciliation ...,1
2,10352,poor-families,The next immediate problem that cropped up was...,1
3,8279,vulnerable,Far more important than the implications for t...,1
4,1164,poor-families,To strengthen child-sensitive social protectio...,1
...,...,...,...,...
8370,8380,refugee,Rescue teams search for survivors on the rubbl...,0
8371,8381,hopeless,The launch of ' Happy Birthday ' took place la...,0
8372,8382,homeless,"The unrest has left at least 20,000 people dea...",0
8373,8383,hopeless,You have to see it from my perspective . I may...,0


In [28]:
trdf1.to_csv("trdf1.csv")

## get the raw test set (Task 1)

In [29]:
rows = [] # will contain par_id, label and text
for idx in range(len(teids)):
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })


In [30]:
len(rows)

2094

In [31]:
tedf1 = pd.DataFrame(rows)
#print(tedf1)

In [32]:
random.shuffle(tedf1.text.tolist())
tedf1.to_csv("tedf1.csv")
#shuffled_tedf1 = tedf1.sample(frac=1).reset_index(drop=True)

In [33]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix


def evaluate_model(true_labels, preds):
    f1 = f1_score(true_labels, preds)
    accuracy = accuracy_score(true_labels, preds)
    precision = precision_score(true_labels, preds)
    recall = recall_score(true_labels, preds)
    conf_matrix = confusion_matrix(true_labels, preds)
    print(f"F1 Score: {f1}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")


    print("Confusion Matrix:")
    print(conf_matrix)



## Baseline for Task 1

In [34]:
print(len(trdf1[trdf1.label==0]))
print(len(trdf1[trdf1.label==1]))
print((len(trdf1[trdf1.label==0]))/len(trdf1[trdf1.label==1]))

7581
794
9.547858942065492


Neg is 10 times more than positive class.

### RoBERTa Negative : Positive = 2:1

- non-pcl : pcl = 2:1
- 46.7% F1 on offical dev set

In [35]:
# downsample negative instances
pcldf = trdf1[trdf1.label==1]
npos = len(pcldf)
#0, limited to twice the number of positive examples
training_set1 = pd.concat([pcldf,trdf1[trdf1.label==0][:npos*2]])

In [36]:
# from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs

# results = []

# for seed in [random.randint(1, 100) for _ in range(10)]:
#     for base in ['roberta', 'deberta']:
#         baseline = base

#         task1_model_args = ClassificationArgs(num_train_epochs=1,
#                                             no_save=True,
#                                             no_cache=True,
#                                             manual_seed = seed,
#                                             overwrite_output_dir=True,
#                                             use_multiprocessing=False,
#                                             use_multiprocessing_for_evaluation=False)
#         if baseline == 'roberta':
#             task1_model = ClassificationModel("roberta",
#                                             'roberta-base',
#                                             args = task1_model_args,
#                                             num_labels=2,
#                                             use_cuda=cuda_available)
#         elif baseline == 'deberta':
#             task1_model = ClassificationModel("deberta",
#                                             'microsoft/deberta-base',
#                                             args = task1_model_args,
#                                             num_labels=2,
#                                             use_cuda=cuda_available)
#         else:
#             print("Please enter valid model to take as a baseline")
#         task1_model.train_model(training_set1[['text', 'label']])
#         preds_task1, _ = task1_model.predict(tedf1.text.tolist())
#         true_labels = tedf1.label.tolist()
#         f1 = f1_score(true_labels, preds_task1)
#         results_tuple = (base, seed, f1)
#         results.append(results_tuple)
# results = pd.DataFrame(results, columns=['Model', 'Seed', 'F1'])
# Creating boxplot
# plt.figure(figsize=(3.5, 2.75))
# ax = sns.boxplot(x='F1', y='Model', hue='Model', data=results, dodge=False, legend=False,
#             palette={'roberta': 'lightblue', 'deberta': 'salmon'})
# plt.title('')
# plt.xlabel('F1 Score')
# plt.ylabel('')
# plt.tight_layout()
# for spine in ax.spines.values():
#     spine.set_visible(False)
# ax.tick_params(axis='y', which='both', left=False)  # Remove y-axis tick marks
# plt.savefig('avg_berta_performance.pdf', format='pdf', bbox_inches='tight')
# plt.show()
# torch.cuda.memory_allocated()
# torch.cuda.empty_cache()
# del task1_model
# del preds_task1
# del true_labels
# torch.cuda.memory_allocated()

### <strike>{RoBERTa Negative : Positive = 3:1}</strike>
- change ratio to 3:1
- F1: 52.21%

In [37]:
# training_set2 = pd.concat([pcldf,trdf1[trdf1.label==0][:npos*3]])

# # train model
# task1_model.train_model(training_set2[['text', 'label']])
# # run predictions
# preds_task1, _ = task1_model.predict(tedf1.text.tolist())
# true_labels = tedf1.label.tolist()
# evaluate_model(true_labels, preds_task1)

# Get Train + Val + Test Set

In [38]:
from sklearn.model_selection import train_test_split
train_set, val_set = train_test_split(trdf1, test_size=0.2, random_state=905)
test_set = tedf1

In [39]:
print(train_set.shape)
print(val_set.shape)
print(test_set.shape)

(6700, 4)
(1675, 4)
(2094, 4)


In [40]:
# model_args = ClassificationArgs(
#           num_train_epochs = 4,
#           learning_rate = 2e-5,
#           overwrite_output_dir = True,
#           evaluate_during_training = True,
#           manual_seed = 42,
#           use_early_stopping = True,
#           early_stopping_consider_epochs = True,
#           early_stopping_patience = 3,
#           save_steps = -1 , # Save the model only at the end of training
#           train_batch_size = 16,
#           eval_batch_size = 8
# )


# model = ClassificationModel('electra', 'google/electra-small-discriminator', args=model_args,num_labels=2,
#                                   use_cuda=cuda_available)


# model.train_model(train_set[['text', 'label']], eval_df = val_set[['text', 'label']])

# predictions, raw_outputs = model.predict(test_set['text'].tolist())
# evaluate_model(test_set['label'].tolist(),predictions )

# Q2.c & e

## Data Augumentation - to training set
- synonym_replacement
- random_insertion
- random_deletion
- random_swap

In [42]:
# torch.cuda.memory_allocated()
# torch.cuda.empty_cache()
# del deberta_model
# del predictions_deberta
# torch.cuda.memory_allocated()

In [51]:
from nltk.corpus import wordnet
from googletrans import Translator
import numpy as np
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

import nltk
from nltk.corpus import stopwords

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

def synonym_replacement(sentence, n):
    words = sentence.split()
    new_sentence = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) > 0:
            synonym = random.choice(synonyms)
            new_sentence = [synonym if word == random_word else word for word in new_sentence]
            num_replaced += 1
        if num_replaced >= n:  # replace less or equal to n words
            break
    return ' '.join(new_sentence)

def random_insertion(sentence, n):
    words = sentence.split()
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        if new_synonym:
            words.insert(random.randrange(len(words)), random.choice(new_synonym))
    return ' '.join(words)

def random_deletion(sentence, p=0.3):
    words = sentence.split()
    if len(words) == 1:
        return sentence
    #f p is set to 0.7, there's a 70% chance that each word will be deleted and a 30% chance that it will be kept
    remaining = [word for word in words if random.uniform(0, 1) > p]
    if len(remaining) == 0:
        return random.choice(words)
    return ' '.join(remaining)

def random_swap(sentence, n=2):
    words = sentence.split()
    length = len(words)
    for _ in range(n):
        idx1, idx2 = random.randint(0, length - 1), random.randint(0, length - 1)
        words[idx1], words[idx2] = words[idx2], words[idx1]  # Swap
    return ' '.join(words)



# translator = Translator()

# def safe_translate(text, dest_language):
#     try:
#         # Correcting the parameter name to 'dest' as expected by googletrans
#         return translator.translate(text, dest=dest_language).text
#     except Exception as e:
#         print(f"Error during translation: {e}")
#         return text

# def back_translation_italian(sentence, language='it'):
#     try:
#         translated = safe_translate(sentence, dest_language=language)
#         back_translated = safe_translate(translated, dest_language='en')
#         return back_translated
#     except Exception as e:
#         print(f"Error during back translation: {e}")
#         return sentence

# def back_translation_german(sentence, language='de'):
#     try:
#         translated = safe_translate(sentence, dest_language=language)
#         back_translated = safe_translate(translated, dest_language='en')
#         return back_translated
#     except Exception as e:
#         print(f"Error during back translation: {e}")
#         return sentence


augmented_text = []
augmentation_functions = [
    synonym_replacement,
    random_insertion,
    random_deletion,
    random_swap
    # back_translation_italian,
    # back_translation_german
]


for sentence in train_set['text']:

    # choose an augmentation strategy
    augment_func = random.choice(augmentation_functions)

    # if augment_func == back_translation_italian:
    #     new_sentence = back_translation_italian(sentence)

    # elif augment_func == back_translation_german:
    #     new_sentence = back_translation_german(sentence)

    if augment_func == random_deletion:
        new_sentence = random_deletion(sentence, p=0.3)
    else:
        # n is the number of words to replace or swap
        new_sentence = augment_func(sentence, n=random.randint(1, 3))

    augmented_text.append(new_sentence)

# Add augmented sentences to the training set
train_set_augmented = train_set.copy()
train_set_augmented['text'] = augmented_text

# Combine training set
train_set_processed = pd.concat([train_set, train_set_augmented], ignore_index=True)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [52]:
print(train_set_processed.shape)

(13400, 4)


### Deberta after augumentation

In [53]:
deberta_model_args = ClassificationArgs(
    num_train_epochs = 4,
    learning_rate=2e-5,
    overwrite_output_dir=True,
    evaluate_during_training=True,
    manual_seed=42,
    use_early_stopping=True,
    early_stopping_consider_epochs=True,
    early_stopping_patience=3,
    save_steps=-1,  # Save the model only at the end of training
    train_batch_size=32,
    eval_batch_size=16,
)

deberta_model = ClassificationModel(
    'deberta',
    'microsoft/deberta-base',
    args=deberta_model_args,
    num_labels=2,
    use_cuda=cuda_available
    )


deberta_model.train_model(train_set_processed[['text', 'label']], eval_df=val_set[['text', 'label']])


result, model_outputs, wrong_predictions = deberta_model.eval_model(val_set[['text', 'label']])


print(result)

predictions_deberta, raw_outputs = deberta_model.predict(test_set['text'].tolist())
evaluate_model(test_set['label'].tolist(),predictions_deberta )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

Running Evaluation:   0%|          | 0/105 [00:00<?, ?it/s]

{'mcc': 0.5027671112952792, 'accuracy': 0.9170149253731343, 'f1_score': 0.5442622950819672, 'tp': 83, 'tn': 1453, 'fp': 53, 'fn': 86, 'auroc': 0.877305374164093, 'auprc': 0.5651761132298304, 'eval_loss': 0.5091044293559943}


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/131 [00:00<?, ?it/s]

F1 Score: 0.5698630136986301
Accuracy: 0.9250238777459407
Precision: 0.6265060240963856
Recall: 0.5226130653266332
Confusion Matrix:
[[1833   62]
 [  95  104]]


### BOW after augumentation
tokenize words of length 3 to 10, use unigrams (ngram_range=(1, 1)), limit the features to the 23,000 most frequent words, and exclude English stop words.

Training: The Logistic Regression model is trained with a very high regularization strength (C=1500) and a high iteration limit to ensure convergence.

In [54]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
#reference: https://www.kaggle.com/code/ziedbaklouti/bow-logistic-regression-sklearn?scriptVersionId=33955168&cellId=5

# Preprocess and Vectorize Text Data
vect1 = CountVectorizer(token_pattern=r'\w{3,10}', ngram_range=(1, 1),
                        max_features=23000, stop_words='english')

# Fit the vectorizer on the training data and transform the training data
X_train_vect = vect1.fit_transform(train_set_processed['text'])
# Transform the val data
X_val_vect = vect1.transform(val_set['text'])

#Train a Logistic Regression Model
log_reg1 = LogisticRegression(C=1500, max_iter=10000)

# Fit the model on the vectorized training data
log_reg1.fit(X_train_vect, train_set_processed['label'])

#Make Predictions and Evaluate the Model
y_predicted_val = log_reg1.predict(X_val_vect)
print('------Val Set--------------')
evaluate_model(val_set['label'],y_predicted_val)

# Transform the test data
print('------Test Set--------------')
X_test_vect = vect1.transform(test_set['text'])
y_predicted_test = log_reg1.predict(X_test_vect)
evaluate_model(test_set['label'],y_predicted_test)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neverthele'] not in stop_words.
  warnings.warn(


------Val Set--------------
F1 Score: 0.26811594202898553
Accuracy: 0.8794029850746269
Precision: 0.34579439252336447
Recall: 0.21893491124260356
Confusion Matrix:
[[1436   70]
 [ 132   37]]
------Test Set--------------
F1 Score: 0.2565597667638484
Accuracy: 0.8782234957020058
Precision: 0.3055555555555556
Recall: 0.22110552763819097
Confusion Matrix:
[[1795  100]
 [ 155   44]]


In [55]:
misclassified_indices = np.where(y_predicted_val != val_set['label'])[0]

# first misclassified example
if len(misclassified_indices) > 0:
    example_index = misclassified_indices[0]
    misclassified_text = val_set['text'].iloc[example_index]
    true_label = val_set['label'].iloc[example_index]
    predicted_label = y_predicted_val[example_index]

    print("Example of Misclassified Text:")
    print(f"Text: {misclassified_text}")
    print(f"True Label: {true_label}, Predicted Label: {predicted_label}")
else:
    print("No misclassified examples found.")


Example of Misclassified Text:
Text: As a result , millions of Syrians as well as peoples of other countries in the region have been made both homeless and hopeless .
True Label: 1, Predicted Label: 0


### TD-IDF after after augumentation

uses unigrams (ngram_range=(1, 1)), which may not capture the sentiment expressed through phrases.

Max Features Limitation: Limiting to 1000 max features

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocess and Vectorize Text Data using TF-IDF
vect1 = TfidfVectorizer(ngram_range=(1, 1),max_features=1000, stop_words='english').fit(train_set.text)

# Fit the vectorizer on the training data and transform the training data
X_train_vect = vect1.fit_transform(train_set_processed['text'])
# Transform the validation data
X_val_vect = vect1.transform(val_set['text'])

# Train a Logistic Regression Model
log_reg1 = LogisticRegression(C=1500, max_iter=10000)

# Fit the model on the vectorized training data
log_reg1.fit(X_train_vect, train_set_processed['label'])
print('------Val Set--------------')
# Make Predictions and Evaluate the Model on the validation set
y_predicted_val = log_reg1.predict(X_val_vect)
evaluate_model(val_set['label'],y_predicted_val)
print('------Test Set--------------')

X_test_vect = vect1.transform(test_set['text'])
y_predicted_test = log_reg1.predict(X_test_vect)
evaluate_model(test_set['label'],y_predicted_test)

------Val Set--------------
F1 Score: 0.2807017543859649
Accuracy: 0.8776119402985074
Precision: 0.3448275862068966
Recall: 0.23668639053254437
Confusion Matrix:
[[1430   76]
 [ 129   40]]
------Test Set--------------
F1 Score: 0.27011494252873564
Accuracy: 0.8787010506208214
Precision: 0.31543624161073824
Recall: 0.23618090452261306
Confusion Matrix:
[[1793  102]
 [ 152   47]]


In [57]:
import numpy as np

# Identifying indices of misclassified examples in the validation set
misclassified_indices_val = np.where(y_predicted_val != val_set['label'])[0]

if len(misclassified_indices_val) > 0:
    example_index_val = misclassified_indices_val[0]  # Just choose the first misclassified example for analysis
    misclassified_text_val = val_set['text'].iloc[example_index_val]
    true_label_val = val_set['label'].iloc[example_index_val]
    predicted_label_val = y_predicted_val[example_index_val]

    print("Example of Misclassified Text in Validation Set:")
    print(f"Text: {misclassified_text_val}")
    print(f"True Label: {true_label_val}, Predicted Label: {predicted_label_val}")
else:
    print("No misclassified examples found in the validation set.")


Example of Misclassified Text in Validation Set:
Text: As a result , millions of Syrians as well as peoples of other countries in the region have been made both homeless and hopeless .
True Label: 1, Predicted Label: 0


# Q3

# Prepare submission

In [58]:
!cat dev.txt | head -n 10

cat: dev.txt: No such file or directory


In [59]:
!zip submission.zip dev.txt test.txt

	zip warning: name not matched: dev.txt
	zip warning: name not matched: test.txt

zip error: Nothing to do! (submission.zip)


In [60]:
!cat test.txt | head -n 10

cat: test.txt: No such file or directory
